In [1]:
import requests
import json

import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler

import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
segment_size = 100

# 1727 rows
data = pd.read_csv('./Dataset/STB.csv')

# Calculate the number of segments in the dataset
print("len(data)", len(data))

data['ma_5'] = data['close_price'].ewm(span=3, adjust=False, min_periods=0).mean()

# Xóa bỏ các dòng có giá trị null do moving average
data.dropna(inplace=True)


len(data) 1727


In [3]:
# closedf = data[['trunc_time','ma_5']]
closedf = data[['trunc_time','close_price','ma_5']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)

      trunc_time  close_price          ma_5
0     2016-01-04        12600  12600.000000
1     2016-01-05        12300  12450.000000
2     2016-01-06        12800  12625.000000
3     2016-01-07        12600  12612.500000
4     2016-01-08        12600  12606.250000
...          ...          ...           ...
1722  2022-11-21        16700  16748.526427
1723  2022-11-22        16900  16824.263213
1724  2022-11-23        17500  17162.131607
1725  2022-11-24        18200  17681.065803
1726  2022-11-25        18900  18290.532902

[1727 rows x 3 columns]
Shape of close dataframe: (1727, 3)


In [4]:
import copy

close_stock = closedf.copy()
del closedf['trunc_time']

X_data = copy.deepcopy(closedf)
Y_data = copy.deepcopy(closedf)

del X_data['close_price']
del Y_data['ma_5']

scaler=MinMaxScaler(feature_range=(0,1))

X_data=scaler.fit_transform(np.array(X_data).reshape(-1,1))
Y_data=scaler.fit_transform(np.array(Y_data).reshape(-1,1))

print(X_data, X_data.shape)
print(Y_data, Y_data.shape)

[[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.34368504]
 [0.36209637]
 [0.38371974]] (1727, 1)
[[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.35726795]
 [0.38178634]
 [0.40630473]] (1727, 1)


In [5]:
del closedf['close_price']
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

print(closedf.shape)

(1727, 1)


In [6]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
# train_data

train_data:  (1381, 1)
test_data:  (346, 1)


In [7]:

def create_dataset(data_set):
    training_size=int(len(data_set)*0.8)
    train_data,test_data=data_set[0:training_size,:],data_set[training_size:len(data_set),:1]
    print("train_data: ", train_data.shape)
    print("test_data: ", test_data.shape)
    return train_data,test_data

In [8]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10

X_train, X_test = create_dataset(X_data)
y_train, y_test = create_dataset(Y_data)
print("X_train: ", X_train)
print("y_train: ", y_train)
print("X_test: ", X_test)
print("y_test", y_test)


train_data:  (1381, 1)
test_data:  (346, 1)
train_data:  (1381, 1)
test_data:  (346, 1)
X_train:  [[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.82183932]
 [0.79453479]
 [0.76846483]]
y_train:  [[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.80210158]
 [0.76357268]
 [0.73905429]]
X_test:  [[0.76075172]
 [0.73383373]
 [0.74521013]
 [0.74468948]
 [0.72757657]
 [0.72966385]
 [0.7262726 ]
 [0.72723791]
 [0.74013826]
 [0.74658844]
 [0.75690935]
 [0.7585219 ]
 [0.766424  ]
 [0.78101878]
 [0.77767244]
 [0.78486905]
 [0.80354599]
 [0.81998028]
 [0.81489276]
 [0.81944482]
 [0.81994689]
 [0.81665002]
 [0.80080993]
 [0.79466384]
 [0.80223453]
 [0.80335895]
 [0.80037324]
 [0.79888038]
 [0.76886367]
 [0.73079388]
 [0.71619387]
 [0.71066783]
 [0.70346991]
 [0.70341887]
 [0.71403709]
 [0.70958944]
 [0.70470467]
 [0.70847114]
 [0.71656322]
 [0.70996553]
 [0.70755365]
 [0.70368678]
 [0.69643148]
 [0.68836893]
 [0.68345068]
 [0.68365249]
 [0.68464038]
 [0.69045619]
 [0.68804222]
 [0.68506128]
 [0

In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=26, min_samples_leaf=2, min_samples_split=10,n_estimators=160, max_features=1)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=26, max_features=1, min_samples_leaf=2,
                      min_samples_split=10, n_estimators=160)

In [10]:
# Lets Do the prediction 

RF_train_predict=rf.predict(X_train)
RF_test_predict=rf.predict(X_test)

RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1381, 1)
Test data prediction: (346, 1)


In [11]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [12]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  160.89899941548643
Train data MSE:  25888.488012904705
Test data MAE:  102.93956924564785
-------------------------------------------------------------------------------------
Test data RMSE:  896.9806652235436
Test data MSE:  804574.3137848708
Test data MAE:  655.0401481763953


In [13]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9984339041241814
Test data explained variance regression score: 0.9689199509752102


In [14]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.9984338805328817
Test data R2 score: 0.9683560596531412


In [15]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  0.00011615363006009067
Test data MGD:  0.0010061500590593025
----------------------------------------------------------------------
Train data MPD:  1.6085712754301194
Test data MPD:  27.95781782500727


In [16]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px

print(RF_train_predict.shape)
print(RF_test_predict.shape)
print(closedf.shape)
print(len(close_stock))

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[0:len(RF_train_predict), :] = RF_train_predict

print("Train predicted data: ", trainPredictPlot)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(RF_train_predict):len(closedf), :] = RF_test_predict

print("Test predicted data: ", testPredictPlot.shape)

close_stock['Predictions']=testPredictPlot
close_stock['Stock']='STB'
close_stock['Model']='SkLearn'
close_stock['Method']='MV'
close_stock.to_csv('./output/STB_SkLearn_MV.csv', index=False)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['trunc_time'],
                       'original_close': close_stock['ma_5'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(1381, 1)
(346, 1)
(1727, 1)
1727
Train predicted data:  [[12727.04361856]
 [12492.58328802]
 [12845.30004835]
 ...
 [           nan]
 [           nan]
 [           nan]]
Test predicted data:  (1727, 1)
